In [13]:
import pandas as pd
import numpy as np

### Feature generation

In [28]:
df_final = pd.read_csv('../datasets/df_combined.csv', parse_dates=["time"])

df_final['time'] = pd.to_datetime(df_final['time'], utc=True)

print(df_final['time'].dtype)


datetime64[ns, UTC]


In [29]:
# df_final.set_index("time", inplace=True)

df_final['hour'] = df_final['time'].dt.hour
df_final['weekday'] = df_final['time'].dt.weekday
df_final['month'] = df_final['time'].dt.month 

In [30]:
# # Ensure 'hour' and 'weekday' columns are created
# df_final['hour'] = df_final['time'].dt.hour
# df_final['weekday'] = df_final['time'].dt.weekday

# Define conditions and values for 'business hour' and 'weekday_category' in one go

# Business hour conditions and values
business_conditions = [
    ((df_final['hour'] > 8) & (df_final['hour'] < 14)) | ((df_final['hour'] > 17) & (df_final['hour'] < 21)),
    (df_final['hour'] >= 14) & (df_final['hour'] <= 17)
]
business_values = [2, 1]

# Weekday category conditions and values
weekday_conditions = [
    (df_final['weekday'] == 6),  # Sunday
    (df_final['weekday'] == 5)   # Saturday
]
weekday_values = [2, 1]

# Assign 'business hour' and 'weekday_category' using np.select
df_final['business hour'] = np.select(business_conditions, business_values, default=0)
df_final['weekday_category'] = np.select(weekday_conditions, weekday_values, default=0)


In [33]:
# Define the list of cities
# cities = ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']

population =  5179243 + 987000 + 6155116 + 1305342 + 1645342

weights = [
5179243 / population,
987000 / population,
6155116 / population,
1305342 / population,
1645342 / population
]
# Calculate the temperature range for each city in a vectorized way
for label in range(5):
    # Compute the absolute difference and store it in a new column
    df_final['temp_range_{}'.format(label)] = abs(df_final['temp_max_{}'.format(label)] - df_final['temp_min_{}'.format(label)])


cities_weights = {label: weight for label, weight in enumerate(weights)}

df_final['temp_weighted'] = sum(df_final[f'temp_{label}'] * weight for label, weight in enumerate(weights))

print(df_final.columns)

Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil gas', 'generation fossil hard coal',
       'generation fossil oil', 'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation nuclear',
       'generation other', 'generation other renewable', 'generation solar',
       'generation waste', 'generation wind onshore', 'total load actual',
       'price day ahead', 'price actual', 'temp_0', 'temp_min_0', 'temp_max_0',
       'pressure_0', 'humidity_0', 'wind_speed_0', 'wind_deg_0', 'rain_1h_0',
       'snow_3h_0', 'clouds_all_0', 'weather_id_0', 'temp_1', 'temp_min_1',
       'temp_max_1', 'pressure_1', 'humidity_1', 'wind_speed_1', 'wind_deg_1',
       'rain_1h_1', 'snow_3h_1', 'clouds_all_1', 'weather_id_1', 'temp_2',
       'temp_min_2', 'temp_max_2', 'pressure_2', 'humidity_2', 'wind_speed_2',
       'wind_deg_2', 'rain_1h_2', 'snow_3h_2

### Feature selection

In [34]:
def multivariate_data(dataset, target, start_index, end_index, 
                      history_size, target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size


    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.appennd(dataset[indices])

        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i:i + target_size])

    return np.array(data), np.array(labels)

In [35]:
train_end_idx = 27048
cv_end_idx = 31056
test_end_idx = 35064

In [ ]:
# X = df_final[df_final.columns.drop('price actual')].values
# y = df_final['price actual'].values

# y = y.reshape(-1, 1)